* Grab tweets corresponding to tweet_ids from CoAID data 
* To view a tweets on browser, use twitter.com/anyuser/status/TWEET_ID

In [ ]:
!pip install tweepy --upgrade

  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
import pandas as pd
import tweepy as tw

In [ ]:
# mount google drive 
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# authenticate to Twitter
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)
# to check rate limit: api.rate_limit_status()["resources"]

In [ ]:
def fetch_tweets(tweet_id):
  try:
    return api.get_status(tweet_id, tweet_mode='extended').full_text
  except:
    return None

def add_text(filename, directory, api):

  df_tweets = pd.read_csv(directory + filename)

  # remove duplicate tweet ids 
  df_tweets.drop_duplicates("tweet_id", inplace = True)

  # add tweet text
  df_tweets["tweets"] = df_tweets["tweet_id"].apply(fetch_tweets)

  # remove empty tweets
  df_tweets.dropna(inplace = True)

  # save in new file ending in "_with_text.csv"
  new_filename = filename[:-4]+"_with_text.csv"
  df_tweets.to_csv(directory + new_filename, index = False)

def add_text_batch(filename, directory, api):
  # uses "statuses_lookup" in Twitter API to overcome rate limit

  df_tweets = pd.read_csv(directory + filename)

  # remove duplicate tweet ids 
  df_tweets.drop_duplicates("tweet_id", inplace = True, ignore_index = True)

  size = df_tweets.shape[0]

  df_tweets["tweets"] = ""

  for i in range(0, size, 100):
    start = i
    end = min(i+100, size)
    id_list = df_tweets["tweet_id"][start:end].tolist()
    tweets = api.statuses_lookup(id_list, map_= True, tweet_mode="extended")

    for item in tweets:
      try:
        df_tweets.loc[df_tweets.tweet_id == item.id, "tweets"] = item.full_text
      except:
        pass
        
  # remove empty tweets
  df_tweets.dropna(inplace = True)

  # save in new file ending in "_with_text.csv"
  new_filename = filename[:-4]+"_with_text.csv"
  df_tweets.to_csv(directory + new_filename, index = False)

In [ ]:
directory = '/content/drive/My Drive/DS4A-Project/Data/' 
files = ['05-01-2020/ClaimFakeCOVID-19_tweets.csv',
         '05-01-2020/ClaimRealCOVID-19_tweets.csv',
         '07-01-2020/ClaimFakeCOVID-19_tweets.csv',
         '07-01-2020/ClaimRealCOVID-19_tweets.csv',
         '05-01-2020/NewsFakeCOVID-19_tweets.csv',
         '05-01-2020/NewsRealCOVID-19_tweets.csv',
         '07-01-2020/NewsFakeCOVID-19_tweets.csv',
         '07-01-2020/NewsRealCOVID-19_tweets.csv']

In [ ]:
for filename in files:
  add_text_batch(filename, directory, api)